In [16]:
#pip install skope-rules
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import pandas as pd
from skrules import SkopeRules
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import warnings
warnings.filterwarnings("ignore")
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn import preprocessing

In [17]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

train_col=[ 'Pclass', 'Sex', 'SibSp','Parch', 'Embarked','Survived']
test_col=['Pclass', 'Sex', 'SibSp','Parch', 'Embarked']

feature=['Pclass', 'Sex', 'SibSp','Parch', 'Embarked']


In [22]:
le_sex = preprocessing.LabelEncoder()
le_Embarked = preprocessing.LabelEncoder()
train["Sex"]=le_sex.fit_transform(train["Sex"].astype('str') )
test["Sex"]=le_sex.transform(test["Sex"].astype('str') )

train["Embarked"]=le_Embarked.fit_transform(train["Embarked"].astype('str') )
test["Embarked"]=le_Embarked.transform(test["Embarked"].astype('str') )
test_processed=test[test_col]
train_processed=train[train_col].dropna(axis = 0)

In [23]:
#k-10 fold cross validation-> cv=10
#Random forest
rclf=RandomForestClassifier()
scoring = ['precision_macro', 'recall_macro','f1_macro']
scores = cross_validate(rclf, train_processed.iloc[:,:-1],train_processed.iloc[:,-1], scoring=scoring, cv=10)
pd.DataFrame(scores)

,fit_time,score_time,test_precision_macro,test_recall_macro,test_f1_macro
0,0.142657,0.000000,0.747315,0.727273,0.733297
1,0.136410,0.010034,0.780449,0.733957,0.744253
2,0.131485,0.010055,0.762813,0.724866,0.733799
3,0.130515,0.012283,0.832285,0.839037,0.835185
4,0.130659,0.011002,0.820899,0.824332,0.822497
5,0.128397,0.010122,0.762884,0.756417,0.759199
6,0.129256,0.011299,0.858037,0.790642,0.806438
7,0.122579,0.010071,0.740777,0.695455,0.703204
8,0.131724,0.017081,0.837500,0.781551,0.795402
9,0.126203,0.007103,0.850461,0.781481,0.795402


In [24]:
#k-10 fold cross validation-> cv=10
gnb = GaussianNB()
scoring = ['precision_macro', 'recall_macro','f1_macro']
scores = cross_validate(gnb, train_processed.iloc[:,:-1],train_processed.iloc[:,-1], scoring=scoring, cv=10)
pd.DataFrame(scores)


,fit_time,score_time,test_precision_macro,test_recall_macro,test_f1_macro
0,0.000000,0.020859,0.767145,0.761039,0.763655
1,0.001995,0.004010,0.707692,0.716578,0.709720
2,0.001973,0.003990,0.753736,0.736096,0.742029
3,0.002992,0.002991,0.756352,0.769786,0.758058
4,0.001991,0.002982,0.786636,0.797059,0.790094
5,0.002030,0.003954,0.762884,0.756417,0.759199
6,0.002992,0.002991,0.797354,0.800535,0.798830
7,0.002013,0.002975,0.774351,0.771123,0.772623
8,0.001996,0.002992,0.776140,0.765508,0.769775
9,0.002990,0.002993,0.813582,0.806614,0.809626


In [25]:
#k-10 fold cross validation-> cv=10
clf = SkopeRules(max_depth_duplication=None,
                 n_estimators=50,
                 precision_min=0.1,
                 recall_min=0.01,
                 feature_names=feature)
scoring = ['precision_macro', 'recall_macro','f1_macro']
scores = cross_validate(clf, train_processed.iloc[:,:-1],train_processed.iloc[:,-1], scoring=scoring, cv=10)
pd.DataFrame(scores)

,fit_time,score_time,test_precision_macro,test_recall_macro,test_f1_macro
0,2.217250,0.173159,0.216667,0.5,0.302326
1,1.863742,0.072806,0.112360,0.5,0.183486
2,1.842132,0.076533,0.191011,0.5,0.276423
3,1.906529,0.078422,0.252809,0.5,0.335821
4,1.867147,0.070839,0.207865,0.5,0.293651
5,1.817930,0.088452,0.179775,0.5,0.264463
6,1.956745,0.069835,0.219101,0.5,0.304688
7,1.757406,0.062554,0.185393,0.5,0.270492
8,1.747001,0.074476,0.168539,0.5,0.252101
9,1.842129,0.097737,0.185393,0.5,0.270492


In [31]:
#Prepare submission

#Random Forest
rclf=RandomForestClassifier()
rclf.fit(train_processed.iloc[:,:-1],train_processed.iloc[:,-1])
prediction=rclf.predict(test_processed)
radnomforest_prediction=pd.DataFrame()
radnomforest_prediction["PassengerId"]=test["PassengerId"]
radnomforest_prediction["Survived"]=prediction
radnomforest_prediction.to_csv("rf_prediction.csv",index=False)

#Naive Bayes
gnb = GaussianNB()
gnb.fit(train_processed.iloc[:,:-1],train_processed.iloc[:,-1])
nb_prediction=gnb.predict(test_processed)
naive_bayes_prediction=pd.DataFrame()
naive_bayes_prediction["PassengerId"]=test["PassengerId"]
naive_bayes_prediction["Survived"]=nb_prediction
naive_bayes_prediction.to_csv("nb_prediction.csv",index=False)

#skoperule
clf = SkopeRules(max_depth_duplication=None,
                 n_estimators=30,
                 precision_min=0.2,
                 recall_min=0.01,
                 feature_names=feature)
clf.fit(train_processed.iloc[:,:-1],train_processed.iloc[:,-1])
sr_prediction=clf.predict(test_processed)
skope_rule_prediction=pd.DataFrame()
skope_rule_prediction["PassengerId"]=test["PassengerId"]
skope_rule_prediction["Survived"]=sr_prediction
skope_rule_prediction.to_csv("sr_prediction.csv",index=False)